In [1]:
# Refs / Related
# /media/paul/data/DockerVols/spark_pysp_jupy
# /media/paul/code/Python/s3_arrow_parquet_py/docker_spark_pysp_jupy.sh

In [1]:
import os

X2 = 'data_y=2020_m=07_d=01_hr=00_part-00000.gz.parquet'
os.path.exists(X2)

True

In [3]:
# os.getcwd()
# os.path.exists("/home/jovyan/ext_vol/jdbc/postgresql-42.2.18.jar")

# export SPARK_CLASSPATH=/home/jovyan/ext_vol/jdbc

True

In [2]:
from pyspark.sql import SparkSession, SQLContext, Row

# https://stackoverflow.com/questions/34948296/using-pyspark-to-connect-to-postgresql
# https://stackoverflow.com/questions/46919791/pyspark-connection-to-postgres-database-in-ipython-notebook
spark = SparkSession.builder.appName("kiwi_X2"
    ).config("spark.jars", "/home/jovyan/ext_vol/jdbc/postgresql-42.2.18.jar"
    ).config("spark.driver.extraClassPath", "/home/jovyan/ext_vol/jdbc/postgresql-42.2.18.jar"
    ).getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
dfX2 = sqlContext.read.parquet(X2)
print(type(dfX2))

<class 'pyspark.sql.dataframe.DataFrame'>


In [6]:
# can only use SQL syntax if register as temp table -> no longer using SQL syntax
# dfX2.registerTempTable("X2")

In [7]:
dfX2.printSchema()

root
 |-- timestamp: long (nullable = true)
 |-- search_params: struct (nullable = true)
 |    |-- flyFrom_type: string (nullable = true)
 |    |-- seats: struct (nullable = true)
 |    |    |-- adults: integer (nullable = true)
 |    |    |-- children: integer (nullable = true)
 |    |    |-- infants: integer (nullable = true)
 |    |    |-- passengers: integer (nullable = true)
 |    |-- to_type: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- currency_rate: double (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- booking_token: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- infant_price: double (nullable = true)
 |    |    |    |-- infants: integer (nullable = true)
 |    |    |    |-- p: string (nullable = true)
 |    |    |    |-- partner: string (nullable = true)
 |    |    |    |-- passengers: integer (nullable = true)
 |    |    |    |-- sid: st

In [4]:
# VERY GOOD SITE  https://sparkbyexamples.com/spark/explode-spark-array-and-map-dataframe-column/

# if rank and sortorder are the same, we can just use explode rather than posexplode

# WHERE IS IND_SID ?? missing in this particular parquet ?? Need to validate Kiwi schema in pipeline !?
#   one option is spark.read.option("mergeSchema", "true").parquet(path)
#   also nifty utility - pip install parquet-cli

# can't drop "search_params.seats" for some reason

df_preK1_samp = dfX2.limit(1000).selectExpr(
    "*", 
    "from_unixtime(timestamp,'yyyy-MM-dd') AS search_date", 
    "data_list.booking_token.sid AS sid", 
    "data_list.booking_token.sid AS ind_sid", 
    "data_list.booking_token.id AS btid", 
    "data_list.booking_token.p AS p", 
    "data_list.booking_token.partner IN('skytra_user') AS skytra_search", 
    "data_list.route AS routes",
    "ROW_NUMBER() OVER(PARTITION BY data_list.booking_token.sid ORDER BY(data_list.booking_token.sid)) AS rank",
    "posexplode(data) as (sortorder,data_list)"
    ).drop("data","currency","currency_rate","data_list")

df_preK1_samp.printSchema()

root
 |-- timestamp: long (nullable = true)
 |-- search_params: struct (nullable = true)
 |    |-- flyFrom_type: string (nullable = true)
 |    |-- seats: struct (nullable = true)
 |    |    |-- adults: integer (nullable = true)
 |    |    |-- children: integer (nullable = true)
 |    |    |-- infants: integer (nullable = true)
 |    |    |-- passengers: integer (nullable = true)
 |    |-- to_type: string (nullable = true)
 |-- search_date: string (nullable = true)
 |-- sid: string (nullable = true)
 |-- ind_sid: string (nullable = true)
 |-- btid: string (nullable = true)
 |-- p: string (nullable = true)
 |-- skytra_search: boolean (nullable = true)
 |-- routes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- aTimeUTC: long (nullable = true)
 |    |    |-- airline: string (nullable = true)
 |    |    |-- dTimeUTC: long (nullable = true)
 |    |    |-- fare_classes: string (nullable = true)
 |    |    |-- flight_no: integer (nullable = true)
 |   

In [9]:
for i in df_preK1_samp.take(2):
    #print(struct(i))
    print(i.sid + '\n' + i.btid + '\n' + i.p)
    #for c in r.cols:
        #if c.name
        #print(c.name)
    print('\n')

000e5df8-fcc0-4293-a24e-e2633e16638c
06db0b7748480000b16fc567_0|0b770a7c48490000da3c2837_0|1cb20b774856000056aa47f4_0|0b7706db4857000050364060_0
34.99|26.19|34.55|43.11


000e5df8-fcc0-4293-a24e-e2633e16638c
06db0b7748480000b16fc567_0|0b770a7c48490000da3c2837_0|190c093e485600007535d06c_0|093e07f048570000acab80fa_0|07f00b77485700000f06e3a7_0|0b7706db48580000f66a503b_0
34.99|26.19|12.99|14.99|24.99|33.52




In [10]:
df_preK1_samp.count()

129230

In [11]:
df_preK1_samp.filter(
    "search_params.flyFrom_type IN ('anywhere', 'radius') OR search_params.to_type IN ('anywhere', 'radius')"
    ).count()
#.first()[0] to return int

0

In [12]:
df_preK1_samp.filter(
    "(search_params.flyFrom_type NOT IN ('anywhere', 'radius') AND search_params.to_type NOT IN ('anywhere', 'radius') ) AND size(data_list.route) >= 7"
    ).count()


15732

In [5]:
# https://sparkbyexamples.com/spark/spark-sql-array-functions/
# https://mungingdata.com/spark-3/array-exists-forall-transform-aggregate-zip_with/

from pyspark.sql.functions import split

df_pkg_prices1 = df_preK1_samp.filter(
    "(search_params.flyFrom_type NOT IN ('anywhere', 'radius') AND search_params.to_type NOT IN ('anywhere', 'radius') ) AND size(data_list.route) < 7 AND rank = 1"
    ).withColumn("a_btid", split("btid","\\|")
    ).withColumn("a_p", split("p","\\|").cast("array<decimal(10,2)>")
    ).drop("btid","p")


In [6]:
piv = df_pkg_prices1.selectExpr(
    "search_date", 
    "timestamp", 
    "rank"
    ).groupby("search_date"
    ).pivot("timestamp"
    ).avg("rank")

In [19]:
"""
url = 'jdbc:postgresql://host/krat_db'
properties = {'user': 'krat_usr', 'password': 'krat_pwd'}

piv.write.jdbc("jdbc:postgresql:localhost", "public.piv", properties = properties)
"""

Py4JJavaError: An error occurred while calling o112.jdbc.
: org.postgresql.util.PSQLException: Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:303)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:51)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:225)
	at org.postgresql.Driver.makeConnection(Driver.java:465)
	at org.postgresql.Driver.connect(Driver.java:264)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:64)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:48)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedComm

In [10]:
# http://localhost:8080/?pgsql=pg_131&username=krat_usr&db=krat_db&ns=public

piv.write.format("jdbc"
    ).option("url", "jdbc:postgresql://docker_host/krat_db"
    ).option("dbtable", "public.piv"
    ).option("user", "krat_usr"
    ).option("password", "krat_pwd"
    ).save()

Py4JJavaError: An error occurred while calling o104.save.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:315)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:51)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:225)
	at org.postgresql.Driver.makeConnection(Driver.java:465)
	at org.postgresql.Driver.connect(Driver.java:264)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:64)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:48)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCom

In [ ]:
piv.printSchema()

In [ ]:
# https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html

"""

# Loading data from a JDBC source
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

jdbcDF2 = spark.read \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

# Specifying dataframe column data types on read
jdbcDF3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .option("customSchema", "id DECIMAL(38, 0), name STRING") \
    .load()

# Saving data to a JDBC source
jdbcDF.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .save()

jdbcDF2.write \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

# Specifying create table column data types on write
jdbcDF.write \
    .option("createTableColumnTypes", "name CHAR(64), comments VARCHAR(1024)") \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

"""

In [ ]:
df_pkg_prices1.printSchema()

In [ ]:
for r in df_pkg_prices1.take(2):
    print(r)
    print('\n')

In [ ]:
from pyspark.sql.functions import explode, arrays_zip

df_pkg_prices2 = df_pkg_prices1.withColumn("splits", explode( arrays_zip("a_btid","a_p") ))

In [ ]:
for r in df_pkg_prices2.take(2):
    print(r.sid)
    print(r.a_btid)
    print(r.a_p)
    print('\n')

In [ ]:
df_pkg_prices2.printSchema()

In [ ]:
print(df_pkg_prices2.first())

In [ ]:
# take returns a list and is faster than limit, proper predicate pushdown probably
from pyspark.sql.functions import regexp_replace

dfSample = df_pkg_prices2.withColumn("pkg_id", regexp_replace("splits.btid","_(\\d+)$","")).groupBy(["pkg_prices1.ind_sid","pkg_prices1.sid","splits.btid"]).sum("splits.p")

In [ ]:
# from pyspark.sql.functions import regexp_replace

dfSample = df_pkg_prices2.withColumn( 
    "pkg_id", regexp_replace("splits.btid","_(\\d+)$","") 
    ).groupBy(["pkg_prices1.sid","pkg_prices1.ind_sid","pkg_id"]
    ).sum("splits.p")

In [ ]:
dfSample.show(2)

In [ ]:
regex_replace(splits.btid,'_(\\d+)$','') AS pkg_id

In [ ]:
seqTest = sqlContext.sql("SELECT sequence(1,5)")
print(seqTest)

In [ ]:
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType

# SparkContext.parallelize converts Python list to RDD and RDD can be converted to DataFrame object
dat = ['user123']
"""
ddl = List(
    StructField("skytra_user", ArrayType, true)
)
"""
schem = StructType([
    StructField("skytra_user", ArrayType)
])


api_u = spark.createDataFrame(spark.sparkContext.parallelize(dat), schem )